In [171]:
! pip install pytelegrambotapi
! pip install transformers
! pip install fuzzywuzzy
! pip install requests
! pip install bs4
! pip install wikipedia

In [172]:
import telebot
import unicodedata
import wikipedia
import re
import requests
import time
from bs4 import BeautifulSoup
from transformers import pipeline

In [173]:
API_TOKEN = '6842209431:AAFQkn1D4_L7IxhawWTEpKZEyXo9JrfU6NY'
bot = telebot.TeleBot(API_TOKEN)

In [174]:
conversation_state = {}
user_data = {}
nutrition_state = {}
donut_qa_state = {}

In [175]:
menu = {
    "Donut de Chocolate": 5.0,
    "Donut de Morango": 6.0,
    "Donut Glaceado": 4.0,
    "Donut de Doce de Leite": 4.50,
    "Donut de Limão": 5.0,
    "Donut de Maracujá": 7.0,
    "Donut de Framboesa": 8.0
}

In [176]:
def normalize_text(text):
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower()

In [177]:
@bot.message_handler(commands=['start'])
def reply_hi(message):
    chat_id = message.chat.id
    conversation_state[chat_id] = 'aguardando_acao'
    user_data[chat_id] = {'donuts': {}}  # Limpar dados para novo atendimento
    welcome_message = "Olá! Bem-vindo à Donuts & Cia! 😊\n"
    bot.reply_to(message, welcome_message +
                 "Digite 'cardápio' para ver o cardápio e fazer um pedido, 'informações nutricionais' para ver informações sobre um donut ou '/start' para reiniciar o atendimento a qualquer momento.")

In [178]:
@bot.message_handler(func=lambda message: True)
def process_message(message):
    chat_id = message.chat.id
    user_message = normalize_text(message.text)
    state = conversation_state.get(chat_id, 'aguardando_acao')  # Get the current state or default to 'aguardando_acao'

    if user_message == '/start':
        reply_hi(message)
    elif user_message == 'localizacao':
        request_location(message)
    elif user_message == 'informacoes nutricionais':  # Check for nutritional info request at any time
        handle_nutritional_info_request(message)
    elif state == 'aguardando_acao':
        if user_message == 'cardapio':
            show_menu(message)
        else:
            bot.reply_to(message, "Desculpe, não entendi. Digite 'cardápio' para ver o cardápio e fazer um pedido, 'informacoes' para ver informações sobre um donut, ou '/start' para reiniciar o atendimento a qualquer momento.")
    elif state == 'escolhendo_donut':
        choose_donut(message, user_message)
    elif state == 'confirmando_donut':
        confirm_donut(message, user_message)
    elif state == 'definindo_quantidade':
        set_quantity(message, user_message)
    elif state == 'adicionando_mais_donuts':
        add_more_donuts(message, user_message)
    elif state == 'confirmando_pedido':
        confirm_order(message, user_message)


In [179]:
def show_menu(message):
    chat_id = message.chat.id
    menu_text = "\n".join([f"- {donut} (R$ {preco:.2f})" for donut, preco in menu.items()])
    bot.reply_to(message, f"Nosso cardápio de donuts:\n{menu_text}\nQual você gostaria de escolher? 😊\n\n(Para informações nutricionais, digite 'informacoes nutricionais')")  # Added hint
    conversation_state[chat_id] = 'escolhendo_donut'
    if chat_id not in user_data or 'donuts' not in user_data[chat_id]:
        user_data[chat_id] = {'donuts': []}

In [180]:

qa = pipeline(
    "question-answering",
    model="timpal0l/mdeberta-v3-base-squad2",
    tokenizer="timpal0l/mdeberta-v3-base-squad2",
)

# Handlers para perguntas gerais sobre donuts
@bot.message_handler(func=lambda message: normalize_text(message.text) == "perguntas sobre donuts")
def handle_donut_questions_request(message):
    chat_id = message.chat.id
    donut_qa_state[chat_id] = "aguardando_pergunta"
    bot.reply_to(message, "Qual a sua pergunta sobre donuts?")

@bot.message_handler(func=lambda message: donut_qa_state.get(message.chat.id) == "aguardando_pergunta")
def handle_donut_question(message):
    chat_id = message.chat.id
    question = message.text

    try:
        page = wikipedia.page("Donut")
        context = page.content
        answer = qa(context=context, question=question)
        bot.reply_to(message, answer['answer'])
    except wikipedia.exceptions.PageError:
        bot.reply_to(message, "Desculpe, não encontrei informações sobre isso na Wikipedia.")
    except wikipedia.exceptions.DisambiguationError as e:
        options = e.options[:5]
        options_text = "\n".join([f"- {option}" for option in options])
        bot.reply_to(message, f"Sua pergunta é ambígua. Você quis dizer:\n{options_text}")
    finally:
        donut_qa_state[chat_id] = None  # Reseta o estado

# Handlers para informações nutricionais
@bot.message_handler(func=lambda message: normalize_text(message.text) == "informacoes nutricionais")
def handle_nutritional_info_request(message):
    chat_id = message.chat.id
    nutrition_state[chat_id] = "aguardando_donut"
    bot.reply_to(
        message,
        "Para qual donut você gostaria de ver as informações nutricionais? (Digite o nome do donut)",
    )
    bot.register_next_step_handler(message, handle_donut_name)

@bot.message_handler(
    func=lambda message: nutrition_state.get(message.chat.id) == "aguardando_donut"
)
def handle_donut_name(message):
    chat_id = message.chat.id
    donut_name = message.text  # Não normalizar para pesquisa na Wikipedia

    try:
        page = wikipedia.page(donut_name)  # Procura a página específica do donut
        context = page.content
        answer = qa(context=context, question="Informações nutricionais:")
        bot.reply_to(message, answer['answer'])
    except wikipedia.exceptions.PageError:
        bot.reply_to(message, f"Desculpe, não encontrei informações nutricionais para '{donut_name}' na Wikipedia.")
    except wikipedia.exceptions.DisambiguationError as e:
        options = e.options[:5]
        options_text = "\n".join([f"- {option}" for option in options])
        bot.reply_to(message, f"Sua busca é ambígua. Você quis dizer:\n{options_text}")
    finally:
        nutrition_state[chat_id] = None




In [181]:
def choose_donut(message, user_message):
    chat_id = message.chat.id
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
    typed_order = "Quero muito um donut de " + user_message
    results = classifier(typed_order, candidate_labels=list(menu.keys()), hypothesis_template="O sabor do donut é {}.")
    max_score_index = results["scores"].index(max(results["scores"]))
    predicted_donut = results["labels"][max_score_index]
    bot.reply_to(message, f"Você quis dizer {predicted_donut}? Por favor, confirme (sim/não).")


    user_data.setdefault(chat_id, {}).setdefault('donuts', {})
    user_data[chat_id]['donut'] = predicted_donut

    conversation_state[chat_id] = 'confirmando_donut'

In [182]:
def confirm_donut(message, user_message):
    chat_id = message.chat.id
    if user_message == 'sim':
        donut_escolhido = user_data[chat_id]['donut']


        if not isinstance(user_data[chat_id]['donuts'], dict):
            user_data[chat_id]['donuts'] = {}

        user_data[chat_id]['donuts'][donut_escolhido] = None
        bot.reply_to(message, f"Ok, {donut_escolhido} adicionado ao seu pedido. Quantos você gostaria?")
        conversation_state[chat_id] = 'definindo_quantidade'

    elif user_message == 'nao':
        bot.reply_to(message, "Por favor, escolha um donut do nosso cardápio.")
        conversation_state[chat_id] = 'escolhendo_donut'
    else:
        bot.reply_to(message, "Desculpe, não entendi. Por favor, responda com 'sim' ou 'não'.")

In [183]:
def add_more_donuts(message, user_message):
    chat_id = message.chat.id
    if user_message is None:
        bot.reply_to(message, "Deseja adicionar donuts de outros sabores? (sim/não)")
        conversation_state[chat_id] = 'adicionando_mais_donuts'
    elif user_message == 'sim':
        show_menu(message)
    elif user_message == 'nao':
        show_order_confirmation(message)
    else:
        bot.reply_to(message, "Desculpe, não entendi. Por favor, responda com 'sim' ou 'não'.")

In [184]:
def set_quantity(message, user_message):
    chat_id = message.chat.id
    donuts = user_data[chat_id]["donuts"]


    next_donut = next((donut for donut, quantity in donuts.items() if quantity is None), None)

    if user_message is None:

        if next_donut:
            bot.reply_to(message, f"Quantos {next_donut} você gostaria de pedir?")
        else:
            bot.reply_to(message, "Você ainda não escolheu nenhum donut. Digite 'cardapio' para ver as opções.")
            conversation_state[chat_id] = 'aguardando_acao'
            return

        conversation_state[chat_id] = 'definindo_quantidade'
    else:
        try:
            quantidade = int(user_message)
            if quantidade > 0:

                user_data[chat_id]["donuts"][next_donut] = quantidade


                next_donut = next((donut for donut, quantity in donuts.items() if quantity is None), None)

                if next_donut:

                    bot.reply_to(message, f"Quantos {next_donut} você gostaria de pedir?")
                else:

                    add_more_donuts(message, None)
            else:
                bot.reply_to(message, "Por favor, informe uma quantidade válida (maior que zero).")
        except ValueError:
            bot.reply_to(message, "Por favor, informe um número para a quantidade.")

In [185]:
def show_order_confirmation(message):
    chat_id = message.chat.id
    donuts_pedido = user_data[chat_id]["donuts"]

    preco_total = sum(menu[d] * q for d, q in donuts_pedido.items())


    pedido_resumo = "\n".join(
         [f"- {q} {d} (R$ {menu[d]:.2f})" for d, q in donuts_pedido.items()]
    )
    bot.reply_to(
        message,
        f"Perfeito! Seu pedido:\n{pedido_resumo}\nPreço total: R$ {preco_total:.2f}\nConfirma? (sim/não)",
    )
    conversation_state[chat_id] = "confirmando_pedido"

In [186]:
def confirm_order(message, user_message):
    chat_id = message.chat.id
    if user_message == 'sim':
        bot.reply_to(message, "Pedido anotado! Agora, só precisamos do seu endereço.")
        request_location(message)
        conversation_state[chat_id] = 'aguardando_localizacao'
    else:
        bot.reply_to(message, "Pedido cancelado. Para iniciar um novo atendimento, digite '/start'.")
        conversation_state[chat_id] = 'aguardando_acao'
        user_data[chat_id] = {}

In [187]:
from getpass import getpass

In [188]:
api_key = getpass('Enter your API key')

Enter your API key··········


In [189]:
def request_location(message):
    chat_id = message.chat.id
    bot.send_message(
        chat_id,
        "Por favor, compartilhe sua localização para receber o pedido.",
        reply_markup=telebot.types.ReplyKeyboardMarkup(
            resize_keyboard=True, one_time_keyboard=True
        ).add(
            telebot.types.KeyboardButton(
                text="Compartilhar localização", request_location=True
            )
        )
    )
    conversation_state[chat_id] = 'aguardando_localizacao'

In [190]:
def get_address_from_coordinates(latitude, longitude, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad HTTP responses
        data = response.json()
        if data['status'] == 'OK':
            address = data['results'][0]['formatted_address']
            address = address.replace(", Brazil", ", Brasil")  # Remove ", Brazil" from the end of the address
            return address
        else:
            return "Endereço não encontrado"
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return "Endereço não encontrado devido a um erro na API"
    except requests.exceptions.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return "Endereço não encontrado devido a um erro na resposta da API"

In [191]:
def confirm_address(chat_id, endereco, message):
    bot.send_message(chat_id, f"{endereco}")
    if not user_data[chat_id].get('endereco_corrigido', False):
        bot.send_message(chat_id, "Se o endereço estiver correto, responda com 'sim'.\n\nSe for necessário corrigir ou complementar o endereço, copie-o, edite-o e o envie de volta:")
    else:
        bot.send_message(chat_id, "Confirma o endereço de entrega? (sim/não)")
    conversation_state[chat_id] = 'confirmando_endereco'
    bot.register_next_step_handler(message, handle_confirmacao_endereco_message)

In [192]:
@bot.message_handler(
    content_types=['location'],
    func=lambda message: conversation_state.get(message.chat.id) == 'aguardando_localizacao'
)
def handle_location_and_confirm(message):
    chat_id = message.chat.id
    latitude = message.location.latitude
    longitude = message.location.longitude
    endereco = get_address_from_coordinates(latitude, longitude, api_key)
    user_data[chat_id]["endereco"] = endereco

    bot.reply_to(message, f"Localização recebida! Endereço encontrado:")
    confirm_address(chat_id, endereco, message)

In [193]:
@bot.message_handler(func=lambda message: conversation_state.get(message.chat.id) == 'confirmando_endereco')
def handle_confirmacao_endereco_message(message):
    chat_id = message.chat.id
    user_message = normalize_text(message.text)

    if user_message == 'sim':
        endereco = user_data[chat_id]["endereco"]
        donuts_pedido = user_data[chat_id]["donuts"]
        preco_total = sum(menu[d] * q for d, q in donuts_pedido.items())
        pedido_resumo = "\n".join(
             [f"- {q} {d} (R$ {menu[d]:.2f})" for d, q in donuts_pedido.items()]
        )
        bot.reply_to(message, f"Pedido confirmado!\n\nSeu pedido:\n{pedido_resumo}\n\nValor total: R$ {preco_total:.2f}\n\nEndereço de entrega:\n{endereco}\n\nEm breve você receberá seus donuts quentinhos 😋!")
        conversation_state[chat_id] = 'aguardando_acao'
        user_data[chat_id] = {}  # Reset user data for the next order
    elif user_message == '/start':
        reply_hi(message)
    else:
        endereco_correto = message.text
        user_data[chat_id]["endereco"] = endereco_correto
        user_data[chat_id]["endereco_corrigido"] = True
        confirm_address(chat_id, endereco_correto, message)

In [ ]:
bot.polling()